In [1]:
!pip install simpy

### Goal
- a source generate visitors

In [2]:
from simpy import Environment, Process

In [3]:
class Visitor(Process):  
    def __init__(self, env, name):
        super().__init__(env, self.visit())
        self.name = name

    def visit(self): 
        print(f"{self.name}: Here I am at {self.env.now:2.1f}") 
        duration = len(self.name)
        yield env.timeout(duration)
        print(f"{self.name}: I must leave at {self.env.now:2.1f}") 

In [4]:
class BasicVisitorSource(Process):  
    def __init__(self, env, names):
        super().__init__(env, self.generate())
        self.names = names

    def generate(self): 
        for name in self.names:
            yield self.env.timeout(1)
            print(f"start {name} at {self.env.now:2.1f}") 
            Visitor(self.env, name)

In [5]:
sim_duration = 10
env = Environment()
source = BasicVisitorSource(env, ["Alice", "Bob"])
print(f'start at {env.now}')
env.run(until=sim_duration)
print(f'stop at {env.now}')

start at 0
start Alice at 1.0
Alice: Here I am at 1.0
start Bob at 2.0
Bob: Here I am at 2.0
Bob: I must leave at 5.0
Alice: I must leave at 6.0
stop at 10


please note that visitor whow once and leave

events.Event
|
+— events.Timeout
|
+— events.Initialize
|
+— events.Process
|
+— events.Condition
|  |
|  +— events.AllOf
|  |
|  +— events.AnyOf
.
.

generator is an iterator, but it easier to write (example with __iter__ and __next__)
as it is an iterator ou may user next


now add a callback
never -processed but automaticaly triggered

run sim until event occur

until n stands for until timeout(n)
stops when the process is done

until expect any event or condition

run sim until event occur
event.succeed
happy with visits

condition and multiple condition

In [6]:
sim_duration = 10
env = Environment()
source = BasicVisitorSource(env, ["Alice", "Bob"])
print(f'start at {env.now}')
env.run(until=source)
print(f'stop at {env.now}')

start at 0
start Alice at 1.0
Alice: Here I am at 1.0
start Bob at 2.0
Bob: Here I am at 2.0
stop at 2


note that it does not wait until visitors are completed. 
This event ends when all vistors are created

In [7]:
class InfiniteVisitorSource(Process):  
    def __init__(self, env):
        super().__init__(env, self.generate())
        self.count = 0

    def generate(self): 
        while True:
            yield self.env.timeout(1)
            self.count += 1
            name = f"visitor-{self.count:03d}"
            print(f"start {name} at {self.env.now:2.1f}") 
            Visitor(self.env, name)

In [8]:
sim_duration = 15
env = Environment()
source = InfiniteVisitorSource(env)
print(f'start at {env.now}')
env.run(until=sim_duration)
print(f'stop at {env.now}')

start at 0
start visitor-001 at 1.0
visitor-001: Here I am at 1.0
start visitor-002 at 2.0
visitor-002: Here I am at 2.0
start visitor-003 at 3.0
visitor-003: Here I am at 3.0
start visitor-004 at 4.0
visitor-004: Here I am at 4.0
start visitor-005 at 5.0
visitor-005: Here I am at 5.0
start visitor-006 at 6.0
visitor-006: Here I am at 6.0
start visitor-007 at 7.0
visitor-007: Here I am at 7.0
start visitor-008 at 8.0
visitor-008: Here I am at 8.0
start visitor-009 at 9.0
visitor-009: Here I am at 9.0
start visitor-010 at 10.0
visitor-010: Here I am at 10.0
start visitor-011 at 11.0
visitor-011: Here I am at 11.0
visitor-001: I must leave at 12.0
start visitor-012 at 12.0
visitor-012: Here I am at 12.0
visitor-002: I must leave at 13.0
start visitor-013 at 13.0
visitor-013: Here I am at 13.0
visitor-003: I must leave at 14.0
start visitor-014 at 14.0
visitor-014: Here I am at 14.0
stop at 15


In [9]:
class RampupSource(Process):  
    def __init__(self, env, rate=1):
        super().__init__(env, self.generate())
        self.rate = rate
        self.count = 0

    def generate(self): 
        while True:
            yield self.env.timeout(1)
            for i in range(0, self.rate):
                self.count += 1
                visitor = self.create(self.count)
                print(f"start {visitor.name} at {self.env.now:2.1f}") 
     
    def create(self, count):
        return None

class RampupVisitorSource(RampupSource):  
    def __init__(self, env, rate=1):
        super().__init__(env, rate)
     
    def create(self, count):
        name = f"visitor-{count:03d}"
        visitor = Visitor(self.env, name)
        print(f"start {visitor.name} at {self.env.now:2.1f}") 
        return visitor


In [10]:
sim_duration = 15
env = Environment()
source = RampupVisitorSource(env, rate=2)
print(f'start at {env.now}')
env.run(until=sim_duration)
print(f'stop at {env.now}')

start at 0
start visitor-001 at 1.0
start visitor-002 at 1.0
visitor-001: Here I am at 1.0
visitor-002: Here I am at 1.0
start visitor-003 at 2.0
start visitor-004 at 2.0
visitor-003: Here I am at 2.0
visitor-004: Here I am at 2.0
start visitor-005 at 3.0
start visitor-006 at 3.0
visitor-005: Here I am at 3.0
visitor-006: Here I am at 3.0
start visitor-007 at 4.0
start visitor-008 at 4.0
visitor-007: Here I am at 4.0
visitor-008: Here I am at 4.0
start visitor-009 at 5.0
start visitor-010 at 5.0
visitor-009: Here I am at 5.0
visitor-010: Here I am at 5.0
start visitor-011 at 6.0
start visitor-012 at 6.0
visitor-011: Here I am at 6.0
visitor-012: Here I am at 6.0
start visitor-013 at 7.0
start visitor-014 at 7.0
visitor-013: Here I am at 7.0
visitor-014: Here I am at 7.0
start visitor-015 at 8.0
start visitor-016 at 8.0
visitor-015: Here I am at 8.0
visitor-016: Here I am at 8.0
start visitor-017 at 9.0
start visitor-018 at 9.0
visitor-017: Here I am at 9.0
visitor-018: Here I am at 9.0